# Diabetes Prediction
# Pre Exploration Questions?
- **What is Diabetes?**
    
    Diabetes is a metabolic disease that causes high blood sugar. The hormone insulin moves sugar from the blood into your cells to be stored or used for energy. With diabetes, your body either doesn't make enough insulin or can't effectively use the insulin it does make.
    
## Where is this data from?
This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective of the dataset is to dianostically predict whether or not a patient has diabetes, based on certain diagnostic measaurements included in the dataset. Several contraints were places on the selection of these instance from a larger database. In particular, all patients here are females at least 21 years old of Pima Indian heritage.

## What are the features/columns in the datset and what do they mean?
- **Pregnancies**: Number of times a female candidate become pregnant
- **Glucose**: Plasma Glucose concentration a 2 hrs in an oral glucose tolerance test.
    - **normal**: 148 mg/dl
    - **diabetic**: <mark>200 mg/dl</mark>
    - **prediabetes**: 140-199 mg/dl
- **Blood Pressure**: Diastolic Blood Pressure level (mm/Hg)
    - **normal**: <120/80mm/Hg
    - **diabetic**: <140/90mm/Hg (or may be it goal/recommended by doctors)
- **SkinThickness**: Triceps skin fold thickness(mm)
    - **normal**: 2.5mm
    - **diabetic**: ---
- **Insulin**: 2-hours  serum insulin (muU/ml)
- **BMI**: Body mass index (weight in Kg/$\text{(height in m)}^2$)
- **Diabetes Pedigree Function**
- **Age**: Age in years
- **Outcome**: Class variable (0 or 1) 268 of 768 are 1, the others are 0.
    

# What needs to be done?
Build a machine learning model to accurately predict whether or not the patients in the dataset have diabetes or not?



In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("diabetes.csv")
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1
